source /c/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Mon_env/Scripts/activate

garder les top 5 des films par années
faire deux tables une sur les films et une uatre sur les séries

format read_parquet
pd.df.toparquet

Group by 'category' and calculate quantiles
quantiles = df.groupby('category')['value'].quantile([0.25, 0.5, 0.75])
print(quantiles)

In [1]:
import os
import pandas as pd



  Importation et Nettoyage desbase de données

Table Name Basic - Noms des cateurs, actrices et directeurs selon le filtre

In [2]:
#Savoir si le fichier existe ou pas 
fichier1 = 'C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/name.basics.tsv.gz'
print("Fichier trouvé:", os.path.exists(fichier1))

#Importation de la base de données
name_basics = pd.read_csv(fichier1, sep='\t')

# Remplacer les chaînes '\N' par NaN
name_basics.replace('\\N', pd.NA, inplace=True)

# Supprimer les lignes sans nom 
name_basics.dropna(subset=['primaryName','primaryProfession', 'knownForTitles'], inplace=True)

# Suppression des colonnes de dates birthyear et deathyear car trop de valeurs nulles à plus de 90%. 
name_basics.drop(columns=['birthYear', 'deathYear'], inplace=True)

# Supprimer les doublons éventuels s'il y en a
name_basics.drop_duplicates(inplace=True)

"""Je me suis rendue compte que supprimer uniquement les doublons ne serait pas assez. Donc j'ai choisi 
de filtrer dans primaryName et dans primaryProfession ceux qui correspondent uniquement auxprofessions :
acteurs, actrices, directeurs ou réalisateurs."""

# Séparer les professions en liste
name_basics['primaryProfession'] = name_basics['primaryProfession'].str.split(',')

# Garder les lignes où au moins une profession est dans ['actor', 'actress', 'director']
professions_voulues = {'actor', 'actress', 'director'}
name_basics = name_basics[name_basics['primaryProfession'].apply(lambda profs: any(p in professions_voulues for p in profs))]

# Trouver les noms qui apparaissent plus d'une fois
#nom_dupliqués = name_basics['primaryName'].value_counts()
#noms_repetes = nom_dupliqués[nom_dupliqués > 30]

# Afficher ces noms (avec le nombre de fois qu’ils apparaissent)
#print(noms_repetes)

# Voir les lignes complètes pour ces noms
#name_basics = name_basics[name_basics['primaryName'].isin(noms_repetes.index)]


name_basics

Fichier trouvé: True


,nconst,primaryName,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,"[actor, miscellaneous, producer]","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,"[actress, soundtrack, archive_footage]","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,"[actress, music_department, producer]","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,"[actor, writer, music_department]","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,"[writer, director, actor]","tt0050986,tt0069467,tt0050976,tt0083922"
...,...,...,...,...
14374128,nm9993700,Sexy Angel,[actress],tt7523066
14374129,nm9993701,Sanjai Kuriakose,[actor],tt8736744
14374131,nm9993703,James Craigmyle,[actor],"tt11212278,tt6914160,tt10627062,tt6225166"
14374136,nm9993708,Eli Bevins,"[producer, director, writer]","tt36103776,tt11702702,tt14508814,tt36042179"


Table Akas

In [3]:
#Savoir si le fichier existe ou pas 
fichier3 = 'C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/title_akas.tsv'
print("Fichier trouvé:", os.path.exists(fichier3))

#Importation de la base de données
title_akas = pd.read_csv(fichier3, sep='\t')

# Etape de remplacement des valeurs \N par des valeurs nulles
title_akas.replace('\\N', pd.NA, inplace=True)

# Tri sur titre original
title_akas = title_akas[title_akas['isOriginalTitle'] == 1]

#Suppression des doublons
title_akas.drop_duplicates(subset= 'titleId',inplace=True)

# Suppression des colonnes indésirables
title_akas.drop(columns=['isOriginalTitle', 'attributes','ordering','types','language','region'], inplace=True)

# Renommer la colonne titleId
title_akas.rename(columns={'titleId': 'tconst'}, inplace=True)

title_akas

# Remplacer les valeurs manquantes dans types par attributes car celle ci complémentarise la type
#title_akas['types'] = title_akas['types'].fillna(title_akas['attributes'])
#fltrer pour ne gardeer que les films qui ontun titre fr.
#title_akas = title_akas[title_akas['region'] == 'FR']
#title_akas.drop_duplicates(subset= 'titleId',inplace=True)
#Remplacer les valeurs vides par fr dans la colonnes language
#title_akas['language'] = title_akas['language'].fillna('fr')
# Trier uniquement la colonne language sur fr
#title_akas = title_akas[title_akas['isOriginalTitle'] == 1]

Fichier trouvé: True


,tconst,title
0,tt0000001,Carmencita
8,tt0000002,Le clown et ses chiens
16,tt0000003,Pauvre Pierrot
26,tt0000004,Un bon bock
34,tt0000005,Blacksmith Scene
...,...,...
51872906,tt9916848,Episode #3.17
51872914,tt9916850,Episode #3.19
51872922,tt9916852,Episode #3.20
51872930,tt9916856,The Wind


title Ratings

In [4]:
#Savoir si le fichier existe ou pas 
fichier4 = 'C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/title.ratings.tsv.gz'
print("Fichier trouvé:", os.path.exists(fichier4))

#Importation de la base de données
title_rating= pd.read_csv(fichier4, sep='\t')

#Filtrer sur averageRating > 7.
#title_rating = title_rating[title_rating['averageRating']> 7.5 ]

title_rating

Fichier trouvé: True


,tconst,averageRating,numVotes
0,tt0000001,5.7,2155
1,tt0000002,5.5,292
2,tt0000003,6.5,2189
3,tt0000004,5.3,187
4,tt0000005,6.2,2937
...,...,...,...
1564711,tt9916846,5.8,6
1564712,tt9916848,5.8,6
1564713,tt9916850,5.8,6
1564714,tt9916852,5.8,6


Title basics

In [5]:
#Savoir si le fichier existe ou pas

#Importation de la base de données
title_basics= pd.read_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/title.basics.tsv.gz', sep='\t')

#Filtrer sur movie
title_basics = title_basics[title_basics['titleType'] == 'movie']

title_basics = title_basics[title_basics['isAdult'] == 0]

#Suppression \\N
title_basics.replace('\\N', pd.NA, inplace=True)

#Remplacer les valeurs possibles manquantese de primary par original
title_basics['primaryTitle'] = title_basics['primaryTitle'].fillna(title_basics['originalTitle'])

# Remplacer les valeurs non numériques par NaN (au cas où il y a '\N' dans les données)
title_basics['startYear'] = pd.to_numeric(title_basics['startYear'], errors='coerce')
title_basics['runtimeMinutes'] = pd.to_numeric(title_basics['runtimeMinutes'], errors='coerce')

# Filtrer les lignes où startYear >= 1950 et <= 2027
title_basics = title_basics[(title_basics['startYear'] >= 1950) & (title_basics['startYear'] <= 2027)]
title_basics = title_basics[(title_basics['runtimeMinutes'] >= 45) & (title_basics['runtimeMinutes'] <= 250)]

# Genres préférés par les creusois : 
# Reality-TV,Game-Show,Adult,News,Short, Documentary, Musical,Music,Talk-Show
genres_exclus = ['Reality-TV', 'Mystery','Game-Show', 'Biography','Adult', 'News', 'Film-Noir','Sport','Short', 'Documentary', 'Musical', 'Music', 'Talk-Show']

# Supprimer les lignes dont la colonne 'genres' contient un de ces genres
title_basics = title_basics[~title_basics['genres'].str.contains('|'.join(genres_exclus), na=False)]

title_basics.dropna(subset=['primaryTitle', 'originalTitle'], inplace =True)

# Suppression des colonnes non désirées et lignes
title_basics = title_basics.drop(columns=['isAdult', 'titleType','originalTitle', 'endYear'])

# Créer la colonne décennie
title_basics['decennie'] = (title_basics['startYear'] // 10) * 10

title_basics

C:\Users\sirnb\AppData\Local\Temp\ipykernel_15412\4025971233.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics= pd.read_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/title.basics.tsv.gz', sep='\t')
C:\Users\sirnb\AppData\Local\Temp\ipykernel_15412\4025971233.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  title_basics.replace('\\N', pd.NA, inplace=True)


,tconst,primaryTitle,startYear,runtimeMinutes,genres,decennie
21266,tt0021617,Arizona Territory,1950.0,56.0,"Drama,Western",1950.0
21703,tt0022064,Lebbra bianca,1951.0,80.0,Drama,1950.0
30443,tt0030998,You're Only Young Twice,1952.0,81.0,Comedy,1950.0
34061,tt0034683,Dramma sul Tevere,1952.0,88.0,Drama,1950.0
34794,tt0035423,Kate & Leopold,2001.0,118.0,"Comedy,Fantasy,Romance",2000.0
...,...,...,...,...,...,...
11634872,tt9916190,Safeguard,2020.0,95.0,"Action,Adventure,Thriller",2020.0
11634911,tt9916270,Il talento del calabrone,2020.0,84.0,Thriller,2020.0
11634956,tt9916362,Coven,2020.0,92.0,"Drama,History",2020.0
11635040,tt9916538,Kuambil Lagi Hatiku,2019.0,123.0,Drama,2010.0


Title_principale

In [6]:
#Savoir si le fichier existe ou pas
fichier7 = 'C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/title.principals.tsv.gz'
print("Fichier trouvé:", os.path.exists(fichier7))

#Importation de la base de données
title_principale= pd.read_csv(fichier7, sep='\t')

#category acteurs, actrices, directeurs
# catégories choisies = actor pour 21899682, actress  pour 16512253, director pour 7948438, producer pour 6961285
categories_choisies = ['actor', 'actress', 'director']

# Dans le dataframe filtré
title_principale = title_principale[title_principale['category'].isin(categories_choisies)]

# Suppression des colonnes
title_principale = title_principale.drop(columns=['job','characters','ordering'])

title_principale


Fichier trouvé: True


,tconst,nconst,category
1,tt0000001,nm0005690,director
4,tt0000002,nm0721526,director
6,tt0000003,nm0721526,director
12,tt0000004,nm0721526,director
14,tt0000005,nm0443482,actor
...,...,...,...
92371314,tt9916880,nm2676923,actress
92371315,tt9916880,nm2676923,actress
92371316,tt9916880,nm1469295,actress
92371317,tt9916880,nm1469295,actress


TMDB

In [7]:
#Savoir si le fichier existe ou pas
fichier8 = 'C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/tmdb_full.csv'
print("Fichier rencontré:", os.path.exists(fichier8))

#Importation de la base de données
tmdb = pd.read_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/tmdb_full.csv', sep=',')

#Trier sur adult en valeur False
tmdb = tmdb[tmdb['adult'] == False]

# Suppression des colonnes non désirées et lignes
tmdb.drop(columns=['adult','backdrop_path','genres','production_companies_country','tagline','homepage'
                   , 'production_companies_name','production_companies_country','title','budget','revenue','video' ], inplace=True)

# Choix de la langue parlée en fr
tmdb = tmdb[tmdb['spoken_languages'].apply(lambda langs: 'fr' in langs)]

# Transformer release date en format date
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'], format='%Y-%m-%d')

# Filtrer le release date par décennie
# Extraire l'année
tmdb['annee'] = tmdb['release_date'].dt.year

# Créer la colonne décennie
tmdb['decennie'] = (tmdb['annee'] // 10) * 10

tmdb = tmdb[(tmdb['annee'] >= 1950)]

#Renommer imdb_id en tconst
tmdb.rename(columns={'imdb_id': 'tconst'}, inplace=True)


tmdb

Fichier rencontré: True


C:\Users\sirnb\AppData\Local\Temp\ipykernel_15412\2352454734.py:6: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  tmdb = pd.read_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Tables/tmdb_full.csv', sep=',')


,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,runtime,spoken_languages,status,vote_average,vote_count,annee,decennie
2,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,6.700,41,1962.0,1960.0
21,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.556,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,111,"['en', 'ja', 'fr']",Released,7.970,15748,2003.0,2000.0
25,28,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",31.785,/gQB8Y5RCMkv2zwzFHbUJX3kAhvA.jpg,['US'],1979-08-15,147,"['km', 'en', 'fr', 'vi']",Released,8.281,7199,1979.0,1970.0
33,63,tt0114746,en,Twelve Monkeys,"In the year 2035, convict James Cole reluctant...",29.184,/gt3iyguaCIw8DpQZI1LIN5TohM2.jpg,['US'],1995-12-29,129,"['en', 'fr']",Released,7.596,7444,1995.0,1990.0
44,75,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.051,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,106,"['en', 'fr']",Released,6.394,4864,1996.0,1990.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308861,230352,tt0138126,fr,Le temps de mourir,The movie starts with a mysterious girl riding...,0.600,/stSzbq0bszPKyuTqghBVbJlLAFp.jpg,['FR'],1970-07-08,80,['fr'],Released,4.900,7,1970.0,1970.0
308883,230662,tt3010780,fr,Allez Hop!,A swimmer struggles to commit to a high dive.,1.400,/tuMXq0XY9RLPkN2vZ8FSouKwonL.jpg,['FR'],2013-06-07,7,['fr'],Released,0.000,0,2013.0,2010.0
309278,232230,tt1331091,fr,Au loin des villages,Since the conflict in Darfur spread to the eas...,0.600,/uWDtpewt4PrwzCBkffaw9fMENZZ.jpg,[],2008-07-07,75,"['fr', 'it']",Released,0.000,0,2008.0,2000.0
309283,232251,tt0167083,cs,"Co je doma to se počítá, pánové...",Czech comedy from the 70ties.,1.067,/lDwd3ixq5Ou8dFAQEd2xOAESHtE.jpg,['XC'],1980-11-01,84,"['en', 'cs', 'fr', 'de']",Released,6.800,6,1980.0,1980.0


Merge

In [8]:
merge1 = pd.merge(tmdb,title_rating, on = 'tconst', how = 'inner')
#Filtrer sur 
#merge1 = merge1[merge1['averageRating']> 6 ]
#merge1.drop(columns=['vote_average'], inplace=True)
merge1

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,runtime,spoken_languages,status,vote_average,vote_count,annee,decennie,averageRating,numVotes
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,6.700,41,1962.0,1960.0,7.2,2515
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.556,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,111,"['en', 'ja', 'fr']",Released,7.970,15748,2003.0,2000.0,8.2,1249380
2,28,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",31.785,/gQB8Y5RCMkv2zwzFHbUJX3kAhvA.jpg,['US'],1979-08-15,147,"['km', 'en', 'fr', 'vi']",Released,8.281,7199,1979.0,1970.0,8.4,741323
3,63,tt0114746,en,Twelve Monkeys,"In the year 2035, convict James Cole reluctant...",29.184,/gt3iyguaCIw8DpQZI1LIN5TohM2.jpg,['US'],1995-12-29,129,"['en', 'fr']",Released,7.596,7444,1995.0,1990.0,8.0,663664
4,75,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.051,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,106,"['en', 'fr']",Released,6.394,4864,1996.0,1990.0,6.4,252014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17846,230023,tt2371345,en,Holden,"Holden, a young photographer with a short-term...",0.600,NaN,"['FR', 'ES']",2012-05-18,20,"['fr', 'es']",Released,4.700,3,2012.0,2010.0,4.8,71
17847,230352,tt0138126,fr,Le temps de mourir,The movie starts with a mysterious girl riding...,0.600,/stSzbq0bszPKyuTqghBVbJlLAFp.jpg,['FR'],1970-07-08,80,['fr'],Released,4.900,7,1970.0,1970.0,5.7,172
17848,230662,tt3010780,fr,Allez Hop!,A swimmer struggles to commit to a high dive.,1.400,/tuMXq0XY9RLPkN2vZ8FSouKwonL.jpg,['FR'],2013-06-07,7,['fr'],Released,0.000,0,2013.0,2010.0,6.6,7
17849,232251,tt0167083,cs,"Co je doma to se počítá, pánové...",Czech comedy from the 70ties.,1.067,/lDwd3ixq5Ou8dFAQEd2xOAESHtE.jpg,['XC'],1980-11-01,84,"['en', 'cs', 'fr', 'de']",Released,6.800,6,1980.0,1980.0,6.1,226


In [9]:
merge2 = pd.merge(merge1,title_basics, on = 'tconst', how = 'inner')
merge2.drop(columns = ['runtime','decennie_y','annee'], inplace=True)
merge2

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,status,vote_average,vote_count,decennie_x,averageRating,numVotes,primaryTitle,startYear,runtimeMinutes,genres
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",Released,6.700,41,1960.0,7.2,2515,Love at Twenty,1962.0,110.0,"Drama,Romance"
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.556,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",Released,7.970,15748,2000.0,8.2,1249380,Kill Bill: Vol. 1,2003.0,111.0,"Action,Crime,Thriller"
2,75,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.051,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,"['en', 'fr']",Released,6.394,4864,1990.0,6.4,252014,Mars Attacks!,1996.0,106.0,"Comedy,Sci-Fi"
3,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.325,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",Released,7.968,3547,1990.0,8.1,358469,Before Sunrise,1995.0,101.0,"Comedy,Drama,Romance"
4,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.184,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",Released,7.818,2943,2000.0,8.1,301797,Before Sunset,2004.0,80.0,"Drama,Romance"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9872,227543,tt1873586,fr,Nino (Une adolescence imaginaire de Nino Ferrer),NaN,0.601,NaN,['FR'],2012-04-24,['fr'],Released,0.000,0,2010.0,6.5,21,Nino (Une adolescence imaginaire de Nino Ferrer),2011.0,72.0,Romance
9873,227549,tt1930420,fr,Noces éphémères,NaN,0.600,NaN,['FR'],2011-10-08,"['fr', 'fa']",Released,0.000,0,2010.0,6.6,34,Ephemeral Marriage,2011.0,78.0,Drama
9874,227922,tt2442772,es,Barbazul,"Based on the classic fairytale ""La Barbe Bleue...",0.600,/9ZEjSslzoRkblQrPZTXSYpKMRhC.jpg,['BO'],2012-10-11,"['es', 'fr', 'en']",Released,0.000,0,2010.0,6.1,34,Bluebeard,2012.0,98.0,Horror
9875,230352,tt0138126,fr,Le temps de mourir,The movie starts with a mysterious girl riding...,0.600,/stSzbq0bszPKyuTqghBVbJlLAFp.jpg,['FR'],1970-07-08,['fr'],Released,4.900,7,1970.0,5.7,172,The Time to Die,1970.0,80.0,"Crime,Sci-Fi,Thriller"


In [10]:
merge3 = pd.merge(merge2,title_akas, on = 'tconst', how = 'inner')
merge3.drop(columns=['primaryTitle','title'], inplace=True)
merge3

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,status,vote_average,vote_count,decennie_x,averageRating,numVotes,startYear,runtimeMinutes,genres
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",Released,6.700,41,1960.0,7.2,2515,1962.0,110.0,"Drama,Romance"
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.556,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",Released,7.970,15748,2000.0,8.2,1249380,2003.0,111.0,"Action,Crime,Thriller"
2,75,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.051,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,"['en', 'fr']",Released,6.394,4864,1990.0,6.4,252014,1996.0,106.0,"Comedy,Sci-Fi"
3,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.325,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",Released,7.968,3547,1990.0,8.1,358469,1995.0,101.0,"Comedy,Drama,Romance"
4,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.184,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",Released,7.818,2943,2000.0,8.1,301797,2004.0,80.0,"Drama,Romance"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9865,227543,tt1873586,fr,Nino (Une adolescence imaginaire de Nino Ferrer),NaN,0.601,NaN,['FR'],2012-04-24,['fr'],Released,0.000,0,2010.0,6.5,21,2011.0,72.0,Romance
9866,227549,tt1930420,fr,Noces éphémères,NaN,0.600,NaN,['FR'],2011-10-08,"['fr', 'fa']",Released,0.000,0,2010.0,6.6,34,2011.0,78.0,Drama
9867,227922,tt2442772,es,Barbazul,"Based on the classic fairytale ""La Barbe Bleue...",0.600,/9ZEjSslzoRkblQrPZTXSYpKMRhC.jpg,['BO'],2012-10-11,"['es', 'fr', 'en']",Released,0.000,0,2010.0,6.1,34,2012.0,98.0,Horror
9868,230352,tt0138126,fr,Le temps de mourir,The movie starts with a mysterious girl riding...,0.600,/stSzbq0bszPKyuTqghBVbJlLAFp.jpg,['FR'],1970-07-08,['fr'],Released,4.900,7,1970.0,5.7,172,1970.0,80.0,"Crime,Sci-Fi,Thriller"


In [12]:
"""merge4 = pd.merge(merge3,title_principale, on = 'tconst', how = 'inner')
merge4"""

"merge4 = pd.merge(merge3,title_principale, on = 'tconst', how = 'inner')\nmerge4"

In [13]:
"""merge5 = pd.merge(merge4,name_basics, on = 'nconst', how = 'inner')
merge5.drop(columns=['primaryProfession'], inplace=True)
merge5"""

"merge5 = pd.merge(merge4,name_basics, on = 'nconst', how = 'inner')\nmerge5.drop(columns=['primaryProfession'], inplace=True)\nmerge5"

In [14]:
"""df = merge5.copy()
#Fonction utilitaire pour formatter "Nom (id)",
def format_name_id(x):
    return ', '.join([f"{name} ({nconst})" for name, nconst in zip(x['primaryName'], x['nconst'])])

#Créer les colonnes séparées par catégorie,
actors = df[df['category'] == 'actor'].groupby('tconst').apply(lambda x: format_name_id(x)).reset_index(name='actors')
actresses = df[df['category'] == 'actress'].groupby('tconst').apply(lambda x: format_name_id(x)).reset_index(name='actresses')
directors = df[df['category'] == 'director'].groupby('tconst').apply(lambda x: format_name_id(x)).reset_index(name='directors')

#Fusionner sur le titre,
result = actors.merge(actresses, on='tconst', how='outer').merge(directors, on='tconst', how='outer')

#Afficher un aperçu,
result"""

'df = merge5.copy()\n#Fonction utilitaire pour formatter "Nom (id)",\ndef format_name_id(x):\n    return \', \'.join([f"{name} ({nconst})" for name, nconst in zip(x[\'primaryName\'], x[\'nconst\'])])\n\n#Créer les colonnes séparées par catégorie,\nactors = df[df[\'category\'] == \'actor\'].groupby(\'tconst\').apply(lambda x: format_name_id(x)).reset_index(name=\'actors\')\nactresses = df[df[\'category\'] == \'actress\'].groupby(\'tconst\').apply(lambda x: format_name_id(x)).reset_index(name=\'actresses\')\ndirectors = df[df[\'category\'] == \'director\'].groupby(\'tconst\').apply(lambda x: format_name_id(x)).reset_index(name=\'directors\')\n\n#Fusionner sur le titre,\nresult = actors.merge(actresses, on=\'tconst\', how=\'outer\').merge(directors, on=\'tconst\', how=\'outer\')\n\n#Afficher un aperçu,\nresult'

In [15]:
"""merge6 = pd.merge(merge3,result, on = 'tconst', how = 'inner')
merge6"""

"merge6 = pd.merge(merge3,result, on = 'tconst', how = 'inner')\nmerge6"

In [11]:
table = pd.merge(title_principale,name_basics, on = 'nconst', how = 'left') 
table.drop(columns=['nconst','primaryProfession','knownForTitles'], inplace=True)
# Création de groupes par catégorie
actors = table[table['category'] == 'actor'].groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x.dropna())).reset_index(name='actors')
actresses = table[table['category'] == 'actress'].groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x.dropna())).reset_index(name='actresses')
directors = table[table['category'] == 'director'].groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x.dropna())).reset_index(name='directors')

# Fusionner les trois catégories
merged_df = pd.merge(actors, actresses, on='tconst', how='outer')
merged_df = pd.merge(merged_df, directors, on='tconst', how='outer')

# Affichage d’un échantillon
merged_df


,tconst,actors,actresses,directors
0,tt0000001,NaN,NaN,William K.L. Dickson
1,tt0000002,NaN,NaN,Émile Reynaud
2,tt0000003,NaN,NaN,Émile Reynaud
3,tt0000004,NaN,NaN,Émile Reynaud
4,tt0000005,,NaN,NaN
...,...,...,...,...
7893724,tt9916848,"Baris Atay, Huseyin Ali Bagotu, Ahmet Bayaslan...","Pelin Akil, Ilter Akçebe, Zeynep Alkan, Ilgin ...",Hamdi Alkan
7893725,tt9916850,"Baris Atay, Huseyin Ali Bagotu, Ahmet Bayaslan...","Pelin Akil, Ilter Akçebe, Zeynep Alkan, Ilgin ...",Hamdi Alkan
7893726,tt9916852,"Baris Atay, Huseyin Ali Bagotu, Ahmet Bayaslan...","Pelin Akil, Ilter Akçebe, Zeynep Alkan, Ilgin ...",Hamdi Alkan
7893727,tt9916856,"Andreas Demmel, Amit Goldenberg, Ardian Harton...","Leni Wesselman, Anna Fitoussi, Kathrin Knöpfle...",Johan Planefeldt


In [13]:
movie= pd.merge(merge3,merged_df, on = 'tconst', how = 'inner')
movie.drop(columns=['averageRating','numVotes'], inplace=True)
movie

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,status,vote_average,vote_count,decennie_x,startYear,runtimeMinutes,genres,actors,actresses,directors
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",Released,6.700,41,1960.0,1962.0,110.0,"Drama,Romance","Jean-Pierre Léaud, Patrick Auffay, François Da...","Marie-France Pisier, Rosy Varte, Cristina Gaio...","Shintarô Ishihara, Marcel Ophüls, Renzo Rossel..."
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.556,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",Released,7.970,15748,2000.0,2003.0,111.0,"Action,Crime,Thriller","David Carradine, Michael Madsen, Shin'ichi Chi...","Uma Thurman, Daryl Hannah, Lucy Liu, Vivica A....",Quentin Tarantino
2,75,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.051,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,"['en', 'fr']",Released,6.394,4864,1990.0,1996.0,106.0,"Comedy,Sci-Fi","Jack Nicholson, Jack Nicholson, Pierce Brosnan...","Sarah Jessica Parker, Annette Bening, Glenn Close",
3,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.325,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",Released,7.968,3547,1990.0,1995.0,101.0,"Comedy,Drama,Romance","Ethan Hawke, Hanno Pöschl, Karl Bruckschwaiger...","Julie Delpy, Andrea Eckert, Erni Mangold",Richard Linklater
4,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.184,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",Released,7.818,2943,2000.0,2004.0,80.0,"Drama,Romance","Ethan Hawke, Vernon Dobtcheff, Rodolphe Pauly,...","Julie Delpy, Louise Lemoine Torrès, Mariane Pl...",Richard Linklater
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855,227543,tt1873586,fr,Nino (Une adolescence imaginaire de Nino Ferrer),NaN,0.601,NaN,['FR'],2012-04-24,['fr'],Released,0.000,0,2010.0,2011.0,72.0,Romance,"David Prat, Benoît Gruel, Benoît Michel, Manue...","Sarah Coulaud, Lou de Laâge, Anne Hiribarren, ...",Thomas Bardinet
9856,227549,tt1930420,fr,Noces éphémères,NaN,0.600,NaN,['FR'],2011-10-08,"['fr', 'fa']",Released,0.000,0,2010.0,2011.0,78.0,Drama,"Hossein Farzizade, Mohammad Javad Taheri, Dari...","Mahnaz Mohammadi, Clothilde Joulin, Masoume Mo...",Reza Serkanian
9857,227922,tt2442772,es,Barbazul,"Based on the classic fairytale ""La Barbe Bleue...",0.600,/9ZEjSslzoRkblQrPZTXSYpKMRhC.jpg,['BO'],2012-10-11,"['es', 'fr', 'en']",Released,0.000,0,2010.0,2012.0,98.0,Horror,"Jac Avila, Roberto Lopez, Erix Antoine","Veronica Paintoux, Mila Joya, Paola Terán, Eri...",Amy Hesketh
9858,230352,tt0138126,fr,Le temps de mourir,The movie starts with a mysterious girl riding...,0.600,/stSzbq0bszPKyuTqghBVbJlLAFp.jpg,['FR'],1970-07-08,['fr'],Released,4.900,7,1970.0,1970.0,80.0,"Crime,Sci-Fi,Thriller","Bruno Cremer, Jean Rochefort, Billy Kearns, Da...","Anna Karina, Lara Koski",André Farwagi


In [15]:
# Export de la table movie
movie.to_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Table_CSV_export/movie.csv', index=False)